In [1]:
# csaw workflow for ATAC-seq differential accessibility analysis,
# adapted from Jake Reske, MSU 2020, reskejak@msu.edu, https://github.com/reskejak

suppressPackageStartupMessages({library(GenomicRanges)
library(csaw)
library("edgeR")
library(data.table)
library(ggplot2)
library(dplyr)})

In [2]:
# read in replicate narrowPeaks
control1.peaks <- fread("peaks/P258-1_S1_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.tn5.pval0.01.300K.bfilt.narrowPeak.gz", 
                           sep="\t",header=FALSE)[,1:3]
control2.peaks <- fread("peaks/P258-2_S2_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.tn5.pval0.01.300K.bfilt.narrowPeak.gz", 
                           sep="\t",header=FALSE)[,1:3]
control3.peaks <- fread("peaks/P258-3_S3_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.tn5.pval0.01.300K.bfilt.narrowPeak.gz", 
                           sep="\t",header=FALSE)[,1:3]
treat1.peaks <- fread("peaks/P260-1_S9_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.tn5.pval0.01.300K.bfilt.narrowPeak.gz", 
                           sep="\t",header=FALSE)[,1:3]
treat2.peaks <- fread("peaks/P260-2_S10_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.tn5.pval0.01.300K.bfilt.narrowPeak.gz", 
                           sep="\t",header=FALSE)[,1:3]
treat3.peaks <- fread("peaks/P260-3_S11_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.tn5.pval0.01.300K.bfilt.narrowPeak.gz", 
                           sep="\t",header=FALSE)[,1:3]
colnames(control1.peaks) <- c("chrom", "start", "end")
colnames(control2.peaks) <- c("chrom", "start", "end")
colnames(control3.peaks) <- c("chrom", "start", "end")
colnames(treat1.peaks) <- c("chrom", "start", "end")
colnames(treat2.peaks) <- c("chrom", "start", "end")
colnames(treat3.peaks) <- c("chrom", "start", "end")

control1.peaks <- GRanges(control1.peaks)
control2.peaks <- GRanges(control2.peaks)
control3.peaks <- GRanges(control3.peaks)
treat1.peaks <- GRanges(treat1.peaks)
treat2.peaks <- GRanges(treat2.peaks)
treat3.peaks <- GRanges(treat3.peaks)

In [3]:
# interesect b/w bio replicates, union across conditions
treat.peaks <- GenomicRanges::intersect(treat1.peaks, treat2.peaks)
treat.peaks <- GenomicRanges::intersect(treat.peaks,treat3.peaks)
control.peaks <- GenomicRanges::intersect(control1.peaks, control2.peaks)
control.peaks <- GenomicRanges::intersect(control.peaks,control3.peaks)
all.peaks <- GenomicRanges::union(treat.peaks, control.peaks)

In [4]:
# specify paired-end BAMs
pe.bams <- c("bams/P258-1_S1_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.bam",
             "bams/P258-2_S2_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.bam",
             "bams/P258-3_S3_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.bam",
             "bams/P260-1_S9_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.bam",
             "bams/P260-2_S10_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.bam",
             "bams/P260-3_S11_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.bam")

blacklist <- read.table("../../../reference_files/hg38_blacklist_ENCODE_ENCFF356LFX.bed", sep="\t")
colnames(blacklist) <- c("chrom", "start", "end")
blacklist <- GRanges(blacklist)

In [5]:
# define read parameters
standard.chr <- paste0("chr", c(1:22)) # only use standard chromosomes
param <- readParam(max.frag=1000, pe="both", discard=blacklist, restrict=standard.chr)

##############################
# count reads in windows specified by MACS2                                      
peak.counts <- regionCounts(pe.bams, all.peaks, param=param)

##############################
# MACS2 peaks only: filter low abundance peaks
library("edgeR")
peak.abundances <- aveLogCPM(asDGEList(peak.counts)) 
peak.counts.filt <- peak.counts[peak.abundances > -3, ] # only use peaks logCPM > -3
# few or no peaks should be removed; modify as desired

##############################

# get paired-end fragment size distribution
#control1.pe.sizes <- getPESizes("~/BE_paper_2022/data/ATAC/258_v_260_edit_v_WT/P258-1_S1_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.bam")
#control2.pe.sizes <- getPESizes("~/BE_paper_2022/data/ATAC/258_v_260_edit_v_WT/P258-2_S2_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.bam")
#control3.pe.sizes <- getPESizes("~/BE_paper_2022/data/ATAC/258_v_260_edit_v_WT/P258-3_S3_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.bam")

#treat1.pe.sizes <- getPESizes("~/BE_paper_2022/data/ATAC/258_v_260_edit_v_WT/P260-1_S9_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.bam")
#treat2.pe.sizes <- getPESizes("~/BE_paper_2022/data/ATAC/258_v_260_edit_v_WT/P260-2_S10_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.bam")
#treat3.pe.sizes <- getPESizes("~/BE_paper_2022/data/ATAC/258_v_260_edit_v_WT/P260-3_S11_merge_R1.fastq.trim.merged.nodup.no_chrM_MT.bam")
#gc()

# plot
#hist(treat1.pe.sizes$sizes) # repeat for all replicates and conditions



In [6]:
##############################
# count BAM reads in, e.g. 300 bp windows
counts <- windowCounts(pe.bams, width=300, param=param) # set width as desired from the fragment length distribution analyses

# filter uninteresting features (windows) by local enrichment
# local background estimator: 2kb neighborhood
neighbor <- suppressWarnings(resize(rowRanges(counts), width=2000, fix="center")) # change width parameter as desired
wider <- regionCounts(pe.bams, regions=neighbor, param=param) # count reads in neighborhoods
# filter.stat <- filterWindows(counts, wider, type="local") # the filterWindows() function is deprecated and has been replaced by filterWindowsLocal(). This is an archived step.
filter.stat <- filterWindowsLocal(counts, wider)
counts.local.filt <- counts[filter.stat$filter > log2(2),] # threshold of 2-fold increase in enrichment over 2kb neighborhood abundance; change as desired

###############################
# count BAM background bins (for TMM normalization)
binned <- windowCounts(pe.bams, bin=TRUE, width=10000, param=param)

peak.counts.tmm <- peak.counts.filt
peak.counts.tmm <- normFactors(binned, se.out=peak.counts.tmm)
working.windows <- peak.counts.tmm 

In [7]:
y <- asDGEList(working.windows)
colnames(y$counts) <- c("control1", "control2", "control3","treat1", "treat2","treat3")
rownames(y$samples) <- c("control1", "control2", "control3","treat1", "treat2","treat3")
y$samples$group <- c("control", "control","control", "treat", "treat","treat")
design <- model.matrix(~0+group, data=y$samples)
colnames(design) <- c("control", "treat") # CONFIRM THAT THESE COLUMNS CORRECTLY ALIGN!!

In [8]:

# stabilize dispersion estimates with empirical bayes
y <- estimateDisp(y, design)
fit <- glmQLFit(y, design, robust=TRUE)

# testing for differentially-accessible windows
results <- glmQLFTest(fit, contrast=makeContrasts(treat-control, levels=design))
# head(results$table)
rowData(working.windows) <- cbind(rowData(working.windows), results$table) # combine GRanges rowdata with differential statistics
# working.windows@rowRanges
 
# merge nearby windows
# up to "tol" distance apart: 500 bp in this case; max merged window width: 5000 bp
merged.peaks <- mergeWindows(rowRanges(working.windows), tol=500L, max.width=5000L)
# summary(width(merged.peaks$region))
# should merge some peaks; change as desired

# use most significant window as statistical representation for p-value and FDR for merged windows
tab.best <- getBestTest(merged.peaks$id, results$table)
 
# concatenating all relevant statistical data for final merged windows (no redundant columns)
final.merged.peaks <- GRanges(cbind(as.data.frame(merged.peaks$region), results$table[tab.best$rep.test, -4], tab.best[,-c(7:8)]))
 
# sort by FDR
final.merged.peaks <- final.merged.peaks[order(final.merged.peaks@elementMetadata$FDR), ]
final.merged.peaks

write.table(final.merged.peaks, 
            "edit_v_WT_stim_windows_all.txt", 
            sep="\t", quote=F, col.names=T, row.names=F)


GRanges object with 75321 ranges and 9 metadata columns:
        seqnames              ranges strand |     logFC     logCPM         F
           <Rle>           <IRanges>  <Rle> | <numeric>  <numeric> <numeric>
  51947     chr8   49168208-49168325      * |   3.15814 -0.7664801   28.7774
  75268    chr13   64047696-64047716      * |  -5.64667 -1.2357133   29.3465
  66792    chr11   81708005-81708123      * |   1.88425 -0.0476377   21.6997
  36165     chr5 144332480-144332529      * |   3.89552 -1.1906214   24.4731
  45698     chr7   33536270-33536343      * |  -3.15173 -1.0195345   21.3078
    ...      ...                 ...    ... .       ...        ...       ...
  96768     chrY   22981832-22981858      * |         0   -2.60503         0
  96769     chrY   24605630-24605704      * |         0   -2.60503         0
  96770     chrY   24610105-24610281      * |         0   -2.60503         0
  96771     chrY   25059810-25060071      * |         0   -2.60503         0
  96772     chrY   

In [11]:
out_counts <- cbind(as.data.frame(merged.peaks$region),
                            results$fitted.values[tab.best$rep.test,])

peak_ROI <- filter(out_counts, ((seqnames == 'chr12') & (start ==9764776)))

cre4_counts <- peak_ROI %>% select(start, control1, control2,control3,treat1,treat2,treat3) %>% tidyr::pivot_longer(!start, names_to = "group", values_to = "count")

lib_info <- results$samples
lib_info$sample <- rownames(lib_info)
cre4_counts <- merge(data.table(cre4_counts),
                     lib_info,
                     by.x = c("group"),
                     by.y = c("sample"))

cre4_counts <- cre4_counts %>% mutate(group = replace(group, grepl('control', group), "control"),
                                      group = replace(group, grepl('treat', group), "treat"))


cre4_counts$counts_norm <- cre4_counts$count / (cre4_counts$lib.size / 1000000.0)
grouped_counts <- cre4_counts %>% group_by(group) %>% summarise(mean = mean(counts_norm),
                                                                sd = sd(counts_norm))


write.table(grouped_counts,
            'RE4_peak.out',sep='\t',quote=FALSE)



In [22]:
### compute the FDR for peaks in this region
system("tail -n+2 edit_v_WT_stim_windows_all.txt | awk '{OFS=\"\t\"}{print $1,$2,$3,$6,$12}' > edit_v_WT_stim_windows_all.bed")
system('bedtools intersect -a edit_v_WT_stim_windows_all.bed -b ROI.bed > ROI_windows.bed')
windows <- fread('ROI_windows.bed')
windows$FDR <- p.adjust(windows$V5, method = "BH", n = 21)
write.table(windows,'ROI_windows.adjusted.bed',sep='\t',quote=FALSE)

In [23]:
windows

V1,V2,V3,V4,V5,FDR
<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>
chr12,9764776,9765449,-0.64668748,0.001648112,0.03461034
chr12,9751938,9752109,-1.15165853,0.005775780,0.06064569
chr12,9435991,9436078,1.22281002,0.031728652,0.18123094
chr12,9708290,9708440,-1.08209989,0.034594045,0.18123094
chr12,9741861,9742058,-0.74342827,0.043150224,0.18123094
chr12,9951138,9951553,0.85227567,0.135138149,0.47298352
chr12,9455710,9455792,0.74791141,0.178818753,0.51390732
chr12,9798095,9798254,-0.61107678,0.195774216,0.51390732
chr12,9674157,9674343,0.49954120,0.220499024,0.51449772
